# Running a query
In the previous tutorial ([Getting started with FlowClient](01-getting-started-with-flowclient.ipynb)), you learned how to use the FlowClient library to connect to a FlowKit server.

In this tutorial you will learn how to use FlowClient to run a FlowKit query and get the result as a pandas dataframe.

## What is a query?

A FlowKit query is a set of parameters that specify a result that can be calculated from the CDR data within the FlowKit database. FlowClient enables you to specify a query, and ask the FlowKit server to run the query and return the result. Some examples of the kinds of queries that FlowKit supports are:

- `unique_subscriber_counts`: Count the number of unique susbcribers who were active in a location during a specified time period,  
- `location_event_counts`: Count the number of CDR events (calls, SMS, ...) in a location during a specified time period,  
- `total_network_objects`: Count the number of network objects (cells) in a location during a specified time period.  

A full list of available query kinds can be found on the [FlowKit documentation website](https://flowkit.xyz/flowclient/flowclient/aggregates/).

There are some restrictions on the query results you are allowed to get from FlowKit:

- You cannot get results which provide information about individual subscribers - FlowKit will only provide results which are aggregated to groups of subscribers (for example, number of subscribers in a region).  
- Only aggregates of more than 15 subscribers are included in the results (for example, if one region only contains 15 subscribers, the subscriber count for that region will not be included in the output from FlowKit).  
- You can only get the result of a query if your access token allows it. An access token can control access to query kinds (for example, you may be allowed to get the result of a `location_event_counts` query but not a `total_network_objects` query), and also query parameters (for example, you may be allowed to get the unique subscriber counts per region (admin1), but not per district (admin2)).  

## Run a query

We will start this tutorial by getting the result of a `unique_subscriber_counts` query as a pandas DataFrame.

### 1. Import the flowclient library

In [ ]:
import flowclient as fc

### 2. Connect to FlowKit
Follow the steps in the [previous tutorial](01-getting-started-with-flowclient.ipynb) to connect to the FlowKit API.

In [ ]:
token = 

conn = fc.connect(
    url="https://api.flowcloud-ghana.flowminder.org",
    token=token,
)

### 3. Define a query
Let's now define a query that will return the count of unique subscribers per district on 1st January 2016. To do this, we can use the flowclient `unique_subscriber_counts` function. We need to provide 4 parameters:

- `connection`: This is the FlowKit connection `conn` that we created in the previous step,  
- `start_date`: This is the start date of the time interval over which we want to count subscribers. In this case, it's `"2016-01-01"`.
- `end_date`: This is the day _after_ the last date in the time interval (in FlowKit queries, time intervals run from midnight at the beginning of `start_date` to midnight at the beginning of `end_date`). In this case we want to count subscribers on just one day, so we set `end_date = "2016-01-02"` (the day after `start_date`).
- `aggregation_unit`: This parameter specifies the set of locations at which we want to count subscribers. In this case, we want the locations to be districts, so we set `aggregation_unit = "admin2"`. The `aggregation_unit` parameter will be explained in more detail in the ['Geography' tutorial](03-geography.ipynb).  

In [ ]:
subscriber_counts_query = fc.unique_subscriber_counts(
    connection=conn,
    start_date="2016-01-01",
    end_date="2016-01-02",
    aggregation_unit="admin2",
)

We have created an APIQuery object, which represents a query that we can run in FlowKit.  
**Note:** We don't yet have the result of the query. In fact, we haven't yet asked the FlowKit server to run this query.

In [ ]:
type(subscriber_counts_query)

If you need to check which parameters you provided when you defined the query, you can check the `parameters` attribute of the query object.  
(Don't worry about the additional parameters with value `None` - these are optional parameters that are not important for now).

In [ ]:
subscriber_counts_query.parameters

### 4. Get the query result
Now that we have defined our 'unique subscriber counts' query, we can ask the FlowKit server to give us the result of the query. We do this by using the `get_result()` method, which returns the result as a pandas DataFrame by default.

In [ ]:
result_of_subscriber_counts_query = subscriber_counts_query.get_result()

Calling the `get_result` method actually did several things:

1. Sent the query parameters to the FlowKit server, and asked the server to run the query,  
2. Waited until the query had finished running,  
3. Asked FlowKit to send back the result of the query.  

You may have seen a progress bar while waiting for the query result. FlowKit splits each query into sub-queries, and the progress bar indicates how many of the sub-queries have finished running. If you did not see a progress bar, this is usually because the query result has already been calculated. FlowKit caches the results of queries, so that the result doesn't need to be re-calculated if a query is run a second time.

We can now look at the pandas DataFrame `result_of_subscriber_counts_query`, which is the result of our query.

In [ ]:
result_of_subscriber_counts_query

The result DataFrame has two columns:

- 'pcod' is the P-code which uniquely identifies a district,  
- 'value' is the number of unique subscribers who were active in that district on 1st January 2016.  

Now that you have got the result of a query, try going back to step 3 ("Define a query"), and change the `start_date` and `end_date` in the query definition, and then get the result of this modified query. You can choose any dates between 2016-01-01 and 2016-07-04 (these are all the dates that are available in this FlowKit, as you saw in the previous tutorial).

## Long-running queries
Some queries can take a long time to run. If we use `get_result` straight away, we will have to wait until the query finishes running before we can do anything else.

Instead, we can use slightly different workflow which will allow us to run other code while we wait for a query result to be ready:

1. Define a query,  
2. Set the query running (without waiting for the result),  
3. Check the status of the query,  
4. Once the query has finished running, get the result.  

### 1. Define queries

As an example, let's start by defining two new queries: a 'location event counts' query and a 'total network objects' query. We povide the same `start_date`, `end_date` and `aggregation_unit` as in the previous example. The `location_event_counts` and `total_network_objects` query kinds each have an additional parameter (`location_event_counts` queries have a `count_interval` parameter, and `total_network_objects` queries have a `total_by` parameter). In both cases, these parameters specify the time intervals over which results should be aggregated. Here we'll use the `"day"` time interval, which means we'll get one result per district for each day between `start_date` and `end_date` (just one day, in this example).

In [ ]:
events_query = fc.location_event_counts(
    connection=conn,
    start_date="2016-01-01",
    end_date="2016-01-02",
    aggregation_unit="admin2",
    count_interval="day",
)

network_objects_query = fc.total_network_objects(
    connection=conn,
    start_date="2016-01-01",
    end_date="2016-01-02",
    aggregation_unit="admin2",
    total_by="day",
)

Query objects have a `status` property, which will tell us the status of the query(e.g. 'executing' or 'completed'). The status of the two new queries is 'not running', because we haven't asked FlowKit to run them yet:

In [ ]:
events_query.status

In [ ]:
network_objects_query.status

### 2. Run the queries

If we now called `events_query.get_result()`, we would have to wait until `events_query` finished before we could start running `network_objects_query`. Instead, we can use the `run` method to set both queries running without waiting for the results:

In [ ]:
events_query.run()
network_objects_query.run()

If we check the status again, we should find that both queries are now either 'queued', 'executing' or 'completed':

In [ ]:
events_query.status

In [ ]:
network_objects_query.status

When we want to get the results of the queries, we can use `get_result` as we did previously. If a query has already finished, `get_result` will return the result immediately. If the query is still running, `get_result` will show a progress bar, and will wait until the query has finished before returning the result.

In [ ]:
events_result = events_query.get_result()
network_objects_result = network_objects_query.get_result()

In [ ]:
events_result

In [ ]:
network_objects_result

Again, both of these queries have a 'pcod' column that identifies the district, and a 'value' column that contains the value for that district (in `events_result`, the 'value' column is the event count, and in `network_objects_result` the 'value' column is the number of cells). Each result also has a date column (`events_result` has 'date' and `network_objects_result` has 'datetime') - since we asked for results aggregated by day, and the time interval specified in the queries was just one day, the value in the 'date'/'datetime' column is the same for every row.

## Summary

In this tutorial you have:
- defined a Flowkit query,  
- got the result of that query as a pandas DataFrame,  
- used `query.run()` to set a query running without waiting for the result (this is useful for queries that take a long time to run),  
- used `query.status` to check whether a query is running.  

In the next tutorial ([Geography](03-geography.ipynb)), you will learn how to aggregate results to different spatial units, and how to get the geographic boundaries of the spatial aggregation units.